# Импорт библиотек

In [4]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

# Подготовка данных

In [5]:
df = pd.read_csv("../../data/frames_errors.csv", header=None)
df.columns = [
    "block_id",  # ID блока
    "frame_idx",  # ID фрейма в блоке
    "E_mu_Z",  # Факт: Реальный QBER в Z-базисе
    "E_mu_phys_est",  # Альтернативная оценка QBER, основанная на физических параметрах
    "E_mu_X",  # Факт: Реальный QBER в X-базисе
    "E_nu1_X",  # QBER для decoy-состояния nu1 в X-базисе
    "E_nu2_X",  # QBER для decoy-состояния nu2 в X-базисе
    "E_nu1_Z",  # QBER для decoy-состояния nu1 в Z-базисе
    "E_nu2_Z",  # QBER для decoy-состояния nu2 в Z-базисе
    "N_mu_X",  # Количество отправленных сигнальных состояний в X-базисе
    "M_mu_XX",  # Количество принятых сигнальных состояний (отправлено в X, принято в X)
    "M_mu_XZ",  # Количество принятых сигнальных состояний (отправлено в X, принято в Z)
    "M_mu_X",  # Общее количество принятых сигнальных состояний в X-базисе
    "N_mu_Z",  # Количество отправленных сигнальных состояний в Z-базисе
    "M_mu_ZZ",  # Количество принятых сигнальных состояний (отправлено в Z, принято в Z)
    "M_mu_Z",  # Общее количество принятых сигнальных состояний в Z-базисе
    "N_nu1_X",  # Количество отправленных decoy-состояний nu1 в X-базисе
    "M_nu1_XX",
    "M_nu1_XZ",
    "M_nu1_X",
    "N_nu1_Z",
    "M_nu1_ZZ",
    "M_nu1_Z",
    "N_nu2_X",
    "M_nu2_XX",
    "M_nu2_XZ",
    "M_nu2_X",
    "N_nu2_Z",
    "M_nu2_ZZ",
    "M_nu2_Z",
    "nTot",  # Общее количество отправленных состояний
    "bayesImVoltage",  # Напряжение на модуляторе интенсивности
    "opticalPower",  # Оптическая мощность
    "polarizerVoltages[0]",  # Напряжение на контроллере поляризации 0
    "polarizerVoltages[1]",  # Напряжение на контроллере поляризации 1
    "polarizerVoltages[2]",  # Напряжение на контроллере поляризации 2
    "polarizerVoltages[3]",  # Напряжение на контроллере поляризации 3
    "temp_1",  # Температура детектора 1
    "biasVoltage_1",  # Напряжение смещения детектора 1
    "temp_2",  # Температура детектора 2
    "biasVoltage_2",  # Напряжение смещения детектора 2
    "synErr",  # Кол-во ошибок в битах
    "N_EC_rounds",  # Результат: Количество доп. раундов коррекции (индикатор сложности)
    "maintenance_flag",  # Флаг технического обслуживания
    "estimator_name",  # Название алгоритма, который выбрал параметры в реальности
    "f_EC",  # Эффективность коррекции ошибок
    "E_mu_Z_est",  # Предсказание: Оценка QBER, на основе которой были выбраны R, s, p
    "R",  # Параметр: Скорость кода
    "s",  # Параметр: Количество укороченных узлов
    "p",  # Параметр: Количество проколотых узлов
]

In [ ]:
missing_counts = df.isna().sum()
columns_with_missing = missing_counts[missing_counts > 0]
print("Столбцы с пропусками и их количество:")
print(columns_with_missing)

Столбцы с пропусками и их количество:
E_mu_phys_est    448
f_EC             131
dtype: int64


In [8]:
timestamp_counts = df.groupby("block_id")["frame_idx"].nunique()
print("Количество фреймов / Количество рядов")
print(timestamp_counts.value_counts())

Количество фреймов / Количество рядов
frame_idx
399    569
400    251
398      2
390      1
Name: count, dtype: int64


# Catboost R

In [14]:
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Используем тот же набор фичей, что и раньше
features = [
    "bayesImVoltage",
    "opticalPower",
    "polarizerVoltages[0]",
    "polarizerVoltages[1]",
    "polarizerVoltages[2]",
    "polarizerVoltages[3]",
    "temp_1",
    "biasVoltage_1",
    "temp_2",
    "biasVoltage_2",
    "E_mu_phys_est",
    "E_mu_Z_est",
]

# Удаляем строки с пропусками в фичах или в целевой переменной 'R'
df_model = df.dropna(subset=features + ["R", "f_EC"]).copy()

X = df_model[features]
y_raw = df_model["R"]
weights = df_model["f_EC"]

# CatBoost может работать с float-классами, но для чистоты преобразуем их в int метки
le = LabelEncoder()
y = le.fit_transform(y_raw)
print("Классы R и их метки:")
print(dict(zip(le.classes_, le.transform(le.classes_))))


# Разделяем на обучающую и валидационную выборки
X_train, X_val, y_train, y_val, weights_train, weights_val = train_test_split(
    X, y, weights, test_size=0.2, random_state=42, stratify=y
)

# --- 2. Обучение модели классификации ---

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    loss_function="MultiClass",
    eval_metric="Accuracy",
    verbose=100,
    random_state=42,
)

print("\nНачинаем обучение модели классификации R...")
model.fit(
    X_train,
    y_train,
    sample_weight=weights_train,
    eval_set=(X_val, y_val),
)

# --- 3. Оценка модели ---
y_pred_labels = model.predict(X_val).flatten()

# Преобразуем предсказанные метки обратно в значения R для наглядности
y_pred_R = le.inverse_transform(y_pred_labels)
y_val_R = le.inverse_transform(y_val)

Классы R и их метки:
{0.5: 0, 0.55: 1, 0.6: 2, 0.65: 3, 0.7: 4, 0.75: 5, 0.8: 6, 0.85: 7}

Начинаем обучение модели классификации R...
0:	learn: 0.9955571	test: 0.9949397	best: 0.9949397 (0)	total: 79.2ms	remaining: 1m 19s
100:	learn: 0.9985320	test: 0.9986587	best: 0.9986587 (68)	total: 4.35s	remaining: 38.7s
200:	learn: 0.9985462	test: 0.9986587	best: 0.9986740 (185)	total: 8.14s	remaining: 32.4s
300:	learn: 0.9986064	test: 0.9986740	best: 0.9986740 (185)	total: 12.4s	remaining: 28.8s
400:	learn: 0.9986313	test: 0.9986740	best: 0.9986740 (185)	total: 16s	remaining: 23.9s
500:	learn: 0.9986626	test: 0.9986587	best: 0.9986740 (185)	total: 19.7s	remaining: 19.6s
600:	learn: 0.9986863	test: 0.9986282	best: 0.9986740 (185)	total: 23.3s	remaining: 15.4s
700:	learn: 0.9987325	test: 0.9986282	best: 0.9986740 (185)	total: 26.8s	remaining: 11.4s
800:	learn: 0.9987554	test: 0.9985825	best: 0.9986740 (185)	total: 30.4s	remaining: 7.56s
900:	learn: 0.9988069	test: 0.9985825	best: 0.9986740 (185)	

ValueError: continuous is not supported

In [19]:
accuracy = accuracy_score(y_val, y_pred_labels)
print(f"\nТочность (Accuracy) модели на валидационной выборке: {accuracy:.4f}")


Точность (Accuracy) модели на валидационной выборке: 0.9987


In [22]:
print("\nОтчет по классификации:")
print(classification_report(y_val, y_pred_labels, target_names=[str(c) for c in le.classes_]))


Отчет по классификации:
              precision    recall  f1-score   support

         0.5       0.99      1.00      0.99       886
        0.55       0.97      0.84      0.90        67
         0.6       0.00      0.00      0.00         3
        0.65       1.00      0.09      0.17        11
         0.7       0.85      0.99      0.91        74
        0.75       1.00      0.96      0.98      1340
         0.8       1.00      1.00      1.00     32529
        0.85       1.00      1.00      1.00     30699

    accuracy                           1.00     65609
   macro avg       0.85      0.73      0.74     65609
weighted avg       1.00      1.00      1.00     65609



# Catboost s p

In [25]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# --- 1. Подготовка данных для регрессии 's' ---

# Используем фичи, добавив R, так как выбор s зависит от него
features_s_reg = [
    "bayesImVoltage",
    "opticalPower",
    "polarizerVoltages[0]",
    "polarizerVoltages[1]",
    "polarizerVoltages[2]",
    "polarizerVoltages[3]",
    "temp_1",
    "biasVoltage_1",
    "temp_2",
    "biasVoltage_2",
    "E_mu_phys_est",
    "E_mu_Z_est",
    "R",
]

# Удаляем строки с пропусками в нужных колонках
df_model_s_reg = df.dropna(subset=features_s_reg + ["s", "f_EC"]).copy()

X_s_reg = df_model_s_reg[features_s_reg]
y_s_reg = df_model_s_reg["s"]  # Целевая переменная - 's' как число
weights_s_reg = df_model_s_reg["f_EC"]

# Разделяем на обучающую и валидационную выборки
X_train_s_reg, X_val_s_reg, y_train_s_reg, y_val_s_reg, weights_train_s_reg, _ = train_test_split(
    X_s_reg, y_s_reg, weights_s_reg, test_size=0.2, random_state=42
)

# --- 2. Обучение модели регрессии для 's' ---

model_s_reg = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    loss_function="RMSE",
    verbose=100,
    random_state=42,
)

print("\nНачинаем обучение модели регрессии 's'...")
model_s_reg.fit(
    X_train_s_reg,
    y_train_s_reg,
    sample_weight=weights_train_s_reg,
    eval_set=(X_val_s_reg, y_val_s_reg),
)

# --- 3. Оценка модели 's' ---
y_pred_s_float = model_s_reg.predict(X_val_s_reg)

# Округляем предсказания до ближайшего целого, чтобы получить итоговое значение 's'
y_pred_s_int = np.round(y_pred_s_float).astype(int)

# Считаем метрики
rmse = np.sqrt(mean_squared_error(y_val_s_reg, y_pred_s_float))
accuracy = accuracy_score(y_val_s_reg, y_pred_s_int)

print(f"\nКачество регрессии (RMSE) на валидационной выборке: {rmse:.4f}")
print(f"Точность (Accuracy) после округления на валидационной выборке: {accuracy:.4f}")

# Сравним несколько предсказаний
comparison_s_df = pd.DataFrame(
    {
        "Реальное s": y_val_s_reg,
        "Предсказанное s (float)": y_pred_s_float,
        "Предсказанное s (int)": y_pred_s_int,
    }
).head(10)
print("\nПример предсказаний vs реальные значения для 's':")
print(comparison_s_df)


Начинаем обучение модели регрессии 's'...
0:	learn: 529.8240596	test: 536.3814202	best: 536.3814202 (0)	total: 17.2ms	remaining: 17.2s
100:	learn: 43.5501529	test: 45.8226745	best: 45.8226745 (100)	total: 769ms	remaining: 6.84s
200:	learn: 30.8330264	test: 32.4894610	best: 32.4894610 (200)	total: 1.37s	remaining: 5.45s
300:	learn: 28.6818544	test: 30.8065476	best: 30.8065476 (300)	total: 2.04s	remaining: 4.74s
400:	learn: 27.6609326	test: 30.1478023	best: 30.1478023 (400)	total: 2.76s	remaining: 4.12s
500:	learn: 27.0785872	test: 30.0002307	best: 29.9958163 (499)	total: 3.4s	remaining: 3.39s
600:	learn: 26.6387754	test: 29.9349457	best: 29.9336977 (592)	total: 4.06s	remaining: 2.69s
700:	learn: 26.2769063	test: 29.9301133	best: 29.9238773 (694)	total: 4.72s	remaining: 2.01s
800:	learn: 25.9231061	test: 29.9315980	best: 29.9183103 (795)	total: 5.29s	remaining: 1.31s
900:	learn: 25.5658640	test: 29.9593450	best: 29.9166738 (847)	total: 6.01s	remaining: 661ms
999:	learn: 25.2879028	test:

# Submit

In [35]:
start_idx = df[(df["block_id"] == 1489460492) & (df["frame_idx"] == 99)].index[0]
end_idx = df[(df["block_id"] == 1840064900) & (df["frame_idx"] == 101)].index[0]

df_submit = df.loc[start_idx:end_idx].copy()

In [36]:
df_submit["E_mu_Z_est"] = df_submit["E_mu_Z"] * 1.1023

In [39]:
X_submit_R = df_submit[features]

# --- 2. Предсказание R ---

print("Предсказываем R...")
# Предсказываем метки классов
pred_labels_R = model.predict(X_submit_R).flatten()
# Преобразуем метки обратно в значения R
pred_values_R = le.inverse_transform(pred_labels_R)

# Добавляем предсказанный R в датафрейм для сабмита
df_submit["R_pred"] = pred_values_R
print("Предсказание R завершено.")


# --- 3. Подготовка данных для предсказания s ---

# Создаем фичи для предсказания s, используя предсказанный R
# Важно: название колонки 'R' должно совпадать с тем, что было при обучении
X_submit_s = df_submit[features_s_reg].rename(columns={"R_pred": "R"})

# --- 4. Предсказание s ---

print("\nПредсказываем s...")
# Предсказываем s как float
pred_float_s = model_s_reg.predict(X_submit_s)
# Округляем до ближайшего целого
pred_int_s = np.round(pred_float_s).astype(int)

# Добавляем предсказанный s в датафрейм
df_submit["s_pred"] = pred_int_s
print("Предсказание s завершено.")

df_submit["p_pred"] = 4800 - df_submit["s_pred"]

# --- 5. Просмотр результата ---
print("\nРезультат предсказаний на df_submit:")
print(df_submit[["E_mu_Z_est", "R_pred", "s_pred", "p_pred"]].head())

Предсказываем R...
Предсказание R завершено.

Предсказываем s...
Предсказание s завершено.

Результат предсказаний на df_submit:
        E_mu_Z_est  R_pred  s_pred  p_pred
285999    0.027635     0.8    4138     662
286000    0.023380     0.8    3413    1387
286001    0.022366     0.8    3236    1564
286002    0.020988     0.8    2994    1806
286003    0.022244     0.8    3209    1591


In [ ]:
df_submit[["E_mu_Z_est", "R_pred", "s_pred", "p_pred"]].to_csv(
    "submission.csv", header=False, index=False
)